In [ ]:
from datasets import load_dataset

ds = load_dataset("CharlyR/varbench", "tikz", split="benchmark")

ds = ds.select_columns(["id","instruction","code","image_solution","image_input"])

#wt do I do with that huh?